In [1]:
import face_recognition
import os
import cv2
import numpy as np


        
        
    
    

In [2]:
KNOWN_FACES_DIR = 'known_faces'
UNKNOWN_FACES_DIR = 'unknown_faces_test'
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = 'hog'

In [3]:
print('Loading known faces...')
known_faces = []
known_names = []

#get all the folders in cwd
for name in os.listdir(KNOWN_FACES_DIR):
    
    #for each file in the folder
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):
        
        #load the image to face_recognition api
        image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')
        
        #learn how to recognize the given above image
        encoding = face_recognition.face_encodings(image, model=MODEL)
        
        #check if we found a face
        if len(encoding) == 0:
            continue
        
        
        #push the encoded face in the known_face
        known_faces.append(encoding[0])
        
        #push the name of the person in known_names
        known_names.append(name)


print(f' we found total {len(known_faces)} faces\n\n')

Loading known faces...
 we found total 19 faces




In [4]:
print('Processing unknown faces...')
 
#unknown faces has some image 
# UNKNOWN_FACES_DIR = 'unknown_faces_test'
#####################################################################################################
#####################################################################################################
######## define a function that can take in an image and draw a rectangle and write name of the person

def detect_faces_in_unknown_image(image, locations, encodings, names_in_the_image):
    global known_faces, TOLERANCE
    
    image_copy = image.copy()
    
    print(len(names_in_the_image))
    
     # image = detectFacesInUnknownImages(image, locations, encodings) 
    for (top, right, bottom, left), face_encoding in zip(locations, encodings):
        
        ## compare the cur_face encoding and check if it matches to a known face
        matches = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE)
        
        ######### initialize the name to None #########
        name = None
        
        ########## find the best match index using the encondings ##########
        face_distances  = face_recognition.face_distance(known_faces, face_encoding)
        
        ######## find the index of the minimum face distance 
        ####### as this is the best match
        best_match_index = np.argmin(face_distances) # returns index of min Value
        
        ######### check if there is a match,  get the name at best match index
        if matches[best_match_index]:
            name = known_names[best_match_index]
        
        if name:
            names_in_the_image.append(name)
            
        ####### show a rectangle on the face
        top_left = (left, top)
        bottom_right = (right, bottom)
        
        cv2.rectangle(image_copy, top_left, bottom_right, (0, 255, 0), 5)
        
        ###### show the name of the person below the rectangle
        font = cv2.FONT_HERSHEY_COMPLEX
        cv2.putText(image_copy, name, (left + 10, bottom + 15), font, 0.5, (200, 200, 200), 10)
        
        return image_copy, names_in_the_image

Processing unknown faces...


In [ ]:
########################################################
# take each image in unknown_faces
for filename in os.listdir(UNKNOWN_FACES_DIR):
    # print the name of the file
    print(f'Filename {filename}', end='')
    
    #load image file and store in image
    image = face_recognition.load_image_file(f'{UNKNOWN_FACES_DIR}/{filename}', mode="RGB")
    
    #get all the faces in the image
    locations = face_recognition.face_locations(image)
    
    #know how to recognize the image based on locations
    encodings = face_recognition.face_encodings(image, locations)
    
    #convert color channel from bgr2rgb
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    #print the number of faces found 
    print(f', found {len(encodings)} face(s)')
    
    if(len(encodings) == 0):
        continue
    
    names_in_the_image = []
    #make pair of the face_location with corresponding encoding
    
    
#     # image = detectFacesInUnknownImages(image, locations, encodings) 
#     for (top, right, bottom, left), face_encoding in zip(locations, encodings):
        
#         ## compare the cur_face encoding and check if it matches to a known face
#         matches = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE)
        
#         ######### initialize the name to None #########
#         name = None
        
#         ########## find the best match index using the encondings ##########
#         face_distances  = face_recognition.face_distance(known_faces, face_encoding)
        
#         ######## find the index of the minimum face distance 
#         ####### as this is the best match
#         best_match_index = np.argmin(face_distances) # returns index of min Value
        
#         ######### check if there is a match,  get the name at best match index
#         if matches[best_match_index]:
#             name = known_names[best_match_index]
        
#         if name:
#             names_in_the_image.append(name)
            
#         ####### show a rectangle on the face
#         top_left = (left, top)
#         bottom_right = (right, bottom)
        
#         cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 5)
        
#         ###### show the name of the person below the rectangle
#         font = cv2.FONT_HERSHEY_COMPLEX
#         cv2.putText(image, name, (left + 10, bottom + 15), font, 0.5, (200, 200, 200), 10)
        
    new_image, names_in_the_image = detect_faces_in_unknown_image(image, locations, encodings, names_in_the_image)
    
    print(f'total {len(names_in_the_image)} people detected in the image')
    ######### get the name from the names_in_the_image and push the image to that folder ########
    for name in names_in_the_image:
        
        ########## get folder by name #######
#         path = os.path.join(KNOWN_FACES_DIR, name)
        
#         ###### write the image in this path ###########
#         cv2.imwrite(f'{KNOWN_FACES_DIR}/{name}/{filename}', image)
        print(f'{name} ')
#     cv2.imshow('image', image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()